## Lab 7: Heterogenous treatment effects, Experiments

In [1]:
import numpy as np
from datascience import Table
%matplotlib inline

## Part 1. Donor Influence/Corruption, Continued

Let's continue to explore these idea of how money may influence political decisions in a slightly different way.

Imagine a company called InfluenceCo, who needs local governments to make favorable decisions for them to be able to run their business profitably. (Perhaps they are a ride-sharing company who needs cities to allow them to operate.)

We will consider the decisions made by mayors of a large number of cities where InfluenceCo wants to do business. As before, we will assume that Mayors vary in their "pro-business" ideology, but also may make decisions based on political donations (or outright bribes).

InfluenceCo wants Mayors to support a policy that would help their business. To simplify we can assume the policy in question is entirely at the discretion of the Mayor.

First suppose that the mayors are all dedicated public servants who will only support the policy if they think it is good for their city. Let's given them a pro-business ideology that ranges from 0 to 1, and put it in a table.

In [2]:
n_mayors = 2000
mayor_ideol = np.random.rand(n_mayors) 
mayor_dataA = Table().with_column("Ideology", mayor_ideol)

A natural assumption is that those with a higher ideology are more likely to support it. Here we will capture this by assuming that the utility for supporting the policy is:
$$
u_{support} = \text{ideology} - e
$$
where $e$ is a random number between 0 and 1 (use `np.random.rand` for this). The utility to not supporting is zero.

**Question 1.1. Write code to create a variables which correspond to the mayor utility for supporting the policy, and a variable equal to 1 if they choose to support it and 0 otherwise. Add the binary support variable to the `mayor_dataA` table with the name "Support".**

In [3]:
#Code for 1.1

Last week we considered a causal process where donations and politician behavior were confounded by their ideology. Now let's think of a process of reverse causation, where politician behavior causes donations.

Here is a data generating process which captures the idea that politicians are more likely to receive donations when they support policies that donors like. In the code below we can think of .8 as the "cost" to donating.

In [4]:
u_donate = .5*support + np.random.rand(n_mayors)
mayor_dataA = mayor_dataA.with_column("Donate", 1*(u_donate > .8))
mayor_dataA

NameError: name 'support' is not defined

**Question 1.2. Give a short theoretical explanation for why politican support decisions might cause donations which is consistent with this code.**

*Answer to 1.2*

**Question 1.3. Compute the difference in the proportion of mayors who support the policy who receieved donations vs. not. Given this data generating process, does this difference reflect a causal effect of donations on voting behavior, selection bias, or both?**

In [5]:
#Code for 1.3

*Words for 1.3*

Throughout this lab we will be taking many of differences of means, and the table notation for this is a bit repetitive. So let's make a function that computes differences of means more efficiently. This will work for any "treatment" variable that takes on values 0 and 1.

In [ ]:
def diffmean(outcome, treatment, data):
    mean1 = np.mean(data.where(treatment, 1).column(outcome))
    mean0 = np.mean(data.where(treatment, 0).column(outcome))
    return(mean1-mean0)

**Question 1.4. Compute the same difference of means as in part 1.4 using the `diffmean` function. (Hint: make sure to enter the variable names in quotation marks).**

In [ ]:
# Code for 1.4

Something missing in this process is the idea that mayors probably want to receive donations, and so if they anticipate receiving donations if they support a policy that could influence their decisions. (If your answer to 1.3 assumed the previous data generating process captured that you might want to change it!)

Here is one way to capture this idea. Let's create a variable called `b_don=.3` which reflects how much mayors like getting donations. The mayor utility for supporting the policy without a donation is the same as above, but let's now call this `u_support0`. Their utility for supporting the policy if they do get a donation is the utility with no donation plus `b_don`. Here is code for this:

In [ ]:
b_don = .3
u_support0 = mayor_ideol - np.random.rand(n_mayors)
u_support1 = u_support0 + b_don

**Question 1.5. Write code to create two variables and add them to `mayor_dataB`, the first of which indicate if the mayor prefers to support the policy if they don't get a donation (call this "Support0") and the second which indicates if the mayor prefers to support the policy if they do get a donation (call this "Support1").** 

In [ ]:
# Code for 1.5

Now let's suppose the donors can anticipate who will support the policy with a donation, and want to reward politicians who do so (but also consider other factors too). The following code captures this idea (it assumes you named the variable correctly in the previous block!) Again, we are implicitly  assuming the "cost" to donating is .8.

In [ ]:
u_donate2 = .5*mayor_dataB.column("Support1") + np.random.rand(n_mayors)
mayor_dataB = mayor_dataB.with_column("Donate", 1*(u_donate2 > .8))
mayor_dataB

Now that we know the donation behavior, we can figure out how the politicians vote. Recall that one formula which can translate the *potential outcomes* $Y_{0,i}$ and $Y_{1,i}$ and the treatment $D_i$ into the realized outcome is:
$$
Y_i = D_i Y_{1,i} + (1-D_i)Y_{0,i}
$$

Another way to do this which shortens the code a bit is to use the `where` function in numpy. This function takes three arguments: (1) a statement which is true or false, (2) what to return if true, and (3) what to return if false. For example:

In [7]:
np.where(3 > 2, 5, 6)

array(5)

In [8]:
np.where(2 > 3, 5, 6)

array(6)

This is helpful with arrays or columns of a table. For example, suppose there are primary elections going on in two parties, and we have arrays which indicate (1) voters' favorite candidate in the Party A, (2) voters' favorite candidate in party B, and (3) which primary they vote in. We can then predict who they will vote for with the `np.where` function:

In [14]:
Afave = np.array(["Smith", "Jones", "Kim", "Jones"])
Bfave = np.array(["Jackson", "Jackson", "Yang", "Yang"])
party = np.array(["A", "B", "A", "B"])
np.where(party == "A", Afave, Bfave)

array(['Smith', 'Jackson', 'Kim', 'Yang'], dtype='<U7')

**Question 1.6. Use the `np.where` function (or the method from the last lab) to create a variable which indicates the realized support decision, and add it to the table with the name "Support"  (i.e., "Support0" with no donation and "Support1" with a donation)**

In [ ]:
# Code for 1.6

**Question 1.7. What is the causal effect of the donation here? It is the difference between the support choice with a donation versus not, which will not necessarily be the same for each mayor. Write code to create a variable and add it to `mayor_dataB` with the name "Causal".**

In [ ]:
# Code for 1.7

**Question 1.8. What is the Average Treatment Effect of a donation here? It should be a number between 0 and 1, why is that the case?**

In [ ]:
# Code for 1.8

*Words for 1.8*

**Question 1.9 what is the Average Treatment Effect on the Treated?**

In [ ]:
# Code for 1.9

**Question 1.10 Write code to confirm that the Difference of Means is equal to the ATET plus selection bias. Remember you can use the `diffmean` function for both of these!**

In [ ]:
#Code for 1.10

## Part 2: Very Strategic Donors [OPTIONAL]

Let's make a dataset with another potential donation process where donors are even more strategic. Suppose they can identify exactly who will be swayed by their choice, and will only donate to those mayors. As a baseline, let's take the ideology and support potential outcomes from the previous dataset.

In [ ]:
mayor_dataC = mayor_dataB.select(["Ideology", "Support0", "Support1"])
mayor_dataC

**Question 2.1 Add a variable to `mayor_dataC` which indicates whether the donor will choose to donate if they only do so if it will cause a change in support, and they know this! (Hint: this will be the same as one of the calculations you did above.)**

In [ ]:
# Code for 2.1

**Question 2.2 What is the average treatment effect on the treated in this table? Hint: you shouldn't need to do any calculations!**

*Words for 2.2*

**Question 2.3. Add a variable for the realized support, and show that the Diff of Means = ATET + Selection Bias formula works for this table too**

In [ ]:
# Code for 2.3

## Part 3. A Fake Experiment

While it would probably be infeasible and unethical to run an experiment where we get donors to randomize which politicians they give money to, we can see how this would play out hypothetically with our simulated data. 

Let's again use the potential outcomes from the previous simulated data, putting it in a table called mayor_dataR (for "randomized")

In [ ]:
mayor_dataR = mayor_dataB.select(["Ideology", "Support0", "Support1", "Causal"])
mayor_dataR

A simple way to randomly decide who gets a donation is to use the `np.random.binomial` function that we used in lab 3. This function takes three arguments. For example the following can be thought of as flipping 10 "biased" coins that are heads with probability .4, and reporting 1 if heads and 0 if tails.

In [ ]:
np.random.binomial(1, .4, 10)

If we change the first argument, that changes "how many coins" we flip for each simulation, e.g., the following simulates flippling 3 biased coins and counting how many heads there are, and doing this 10 times:

In [ ]:
np.random.binomial(3, .4, 10)

If we increase the second argument, we will tend to get more "heads":

In [ ]:
np.random.binomial(3, .8, 10)

And if we increase the third argument, we do more sets of flips.

In [ ]:
np.random.binomial(3, .8, 20)

**Question 3.1. Write code which randomly says whether each mayor gets a donation or not (think of it as 1 coin flip), where each gets a donation with probability .5. Add this to the `mayor_dataR` table with the name "DonateR".**

In [ ]:
# Code for 3.1

**Question 3.2 Make a variable for the realized support choice with the random donation, and add it to `mayor_dataR` with the name "Support".**

In [ ]:
# Code for 3.2

**Question 3.3 Compute the difference in mean of "Support" among those received a (random) donation versus not, and compare this to the Average Treatment Effect and the Average Treatment Effect on the Treated (hint: use the "Causal" variable).**

In [ ]:
# Code for 3.3


*Words for 3.3*

Now let's repeat the process of process of simulating data with (1) biased donations like in the `mayor_dataB` data and (2) random donations like in the `mayor_dataR`. Here are functions to do so.

In [ ]:
def sim_domB():
    donate = .5*mayor_dataB.column("Support1") + np.random.rand(n_mayors)>.8
    support = donate*mayor_dataB.column("Support1") + (1-donate)*mayor_dataB.column("Support0")
    return(np.mean(support[donate==1])-np.mean(support[donate==0]))

In [ ]:
def sim_domR():
    donate = np.random.binomial(1,.5,n_mayors)
    support = donate*mayor_dataB.column("Support1") + (1-donate)*mayor_dataB.column("Support0")
    return(np.mean(support[donate==1])-np.mean(support[donate==0]))

In [ ]:
sim_domB()

In [ ]:
sim_domR()

**Question 3.4. Run 10,000 simulations with the biased donation (`get_domB`) and the random donation (`get_domR`), and make a histogram of both. Use these to explain the value of experiments with randomized treatment for learning about causal effects.**

In [ ]:
# Code for 3.4

## Part 4. A real Experiment

Now let's see how these ideas play out with some real data. Unlike with the simulations, we can't check whether there isn't any selection bias driven by differences in treatment and control groups, since in "real world" mode we don't get to know the potential outcomes. But with the confidence gained by our simulations, if the treatment is randomized, then we can assume that selection bias is likely to be small. We can also check that the treatment and control groups are otherwise similar.

The example we will use is from <a href="https://www.jstor.org/stable/26379536?seq=1#metadata_info_tab_contents">this</a> paper, which aimed to see if sending letters to political elites can encourage them to get women more involved in leadership positions.

Quoting directly from the article: "We ran a field experiment with the cooperation of a state Republican Party in a state with low levels of women's representation (Utah). Party leaders were concerned that although women comprised about half of the party activists who attended neighborhood caucus meetings, women typically accounted for only 20-25% of the delegates elected from these meetings to attend the state nominating convention. We randomly assigned over 2,000 precinct chairs to receive one of four letters from the state party chair prior to these neighborhood caucus meetings. The treatments were a neutral placebo control (Control), a request to recruit 2-3 women to run as state delegates (Supply), a request to read a letter at the precinct meeting encouraging attendees to elect more women as delegates (Demand), and a request to both recruit women and read the letter (Supply+Demand)."

Note that in this case, like in many real experiments, there is not just one treatment and one control. In sense there are two: supply and demand. Since precinct chairs could get neither, one, or both, there are four "treatment statuses". We will see how these groups differ individually, as well as comparing "supply to no supply" and "demand to no demand".

The outcomes we will look at the the proportion of female delegates and whether there are any female delegates. Here are the variables in the data:
- unique_id: Precinct ID
- "treat": treatment variable, with four possibilities:
(1) 'control': control group
(2) 'supply': supply group; party chair instructed to recruit 2-3 women
(3) 'demand': demand group; party chair reads letter at precinct convention
(4) 'both': a fourth group getting both the supply and demand treatments; party chair instructed to read letter and to recruit 2-3 women
- "prop_sd_fem2014": Outcome: Proportion of 2014 elected state delegates from that precinct who were women
- "sd_onefem2014": 1 if at least one woman was selected; 0 otherwise
- "county": County name in Utah
- "pc_male": 1 if precinct chair is male; 0 otherwise (precinct chair is person who runs precinct meeting, would read letter if assigned to do so, etc.)

In [ ]:
utah = Table.read_table("electing_women.csv")
utah

**Question 4.1. Compute the average proportion of female delegates for the each of the four possible treatments**

In [ ]:
# Code for 4.1

**Question 4.2. Compute how many precincts sent at least one female delegates for the each of the four possible treatments. Interpret the results from this and 4.1.**

In [ ]:
# Code for 4.2

**Question 4.3. If we want to estimate the effect of the supply and demand treatments individually, we can create variables that indicate whether they received the supply treatment (setting aside the demand treatment) and vice versa. Create a variable and add it to the table with the name "supply" which is equal to 1 if "treat" is equal to "supply" or "both", and 0 otherwise. Then create a variable and add it to the table with the name "demand" which is equal to 1 if "treat" is equal to "demand" or "both", and 0 otherwise.**

In [ ]:
# Code for 4.3

**Question 4.4. Compute the difference in means for the proportion of female delegates for those who did and did not receive the demand treatment, and those who did and did not receive the supply treatment. You can use the `diffmean` function since there is now a binary treatment equal to 0 or 1.**

In [ ]:
#Code for 4.4

**Question 4.5. While we can't learn the potential outcomes of what delegates would have been sent with a different treatment, we can check to see if those who received different treatments did happen to be different on other dimensions. One that might matter is whether the precinct chair was a male or female (recall this is in the "pc_male" variable). Use the ``diffmean`` fuction to check if precincts with a male chair were more or less likely to send at least one female delegate.**

In [ ]:
# Code for 4.5

**Question 4.6. Now use the `diffmean` function to check whether precincts who recieved the "supply" teatment were more likely to have a male precinct chair, then do the same for the "demand" treatment.**

In [ ]:
# Code for 4.6

**Question 4.7. If a reader of  this study said "It's hard to say whether encouraging women to run is helpful, because male political leaders are more likely to do so and they also do other things which make women less likely to run", how could the authors respond? (Hint: think about the power of randomization in general, and what you found in the previous questions.)**

*Answer to 4.7*